[View in Colaboratory](https://colab.research.google.com/github/nautilusPrime/cifar10-image-classifier/blob/master/cifar10.ipynb)

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
l2_lambda = 0.0001

In [5]:
# You can modify Model here.
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:],
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3),
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same',
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3),
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3),
                 kernel_initializer='he_uniform', 
                 kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(512,
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes,
                kernel_initializer='glorot_uniform', 
                kernel_regularizer=l2(l2_lambda)))
model.add(Activation('softmax'))

In [6]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

In [7]:
x_train = x_train.astype('float16')
x_test = x_test.astype('float16')
x_train /= 255
x_test /= 255

In [8]:
data_augmentation = False

In [9]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[EarlyStopping(monitor='val_loss', patience=8)])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), samples_per_epoch=len(x_train),
                        epochs=epochs, validation_data=(x_test, y_test), workers=4)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
33600/50000 [===================>..........] - ETA: 23s - loss: 1.9099 - acc: 0.4260

50000/50000 [==============================] - 74s 1ms/step - loss: 1.7845 - acc: 0.4645 - val_loss: 1.5247 - val_acc: 0.5398
Epoch 2/100
20448/50000 [===========>..................] - ETA: 40s - loss: 1.4116 - acc: 0.5962

50000/50000 [==============================] - 72s 1ms/step - loss: 1.3441 - acc: 0.6261 - val_loss: 1.3258 - val_acc: 0.6400
Epoch 3/100
14720/50000 [=======>......................] - ETA: 49s - loss: 1.2363 - acc: 0.6718

50000/50000 [==============================] - 72s 1ms/step - loss: 1.2296 - acc: 0.6803 - val_loss: 1.2545 - val_acc: 0.6773
Epoch 4/100
13088/50000 [======>.......................] - ETA: 51s - loss: 1.1883 - acc: 0.7039

50000/50000 [==============================] - 72s 1ms/step - loss: 1.1926 - acc: 0.7096 - val_loss: 1.2022 - val_acc: 0.7097
Epoch 5/100
12768/50000 [======>.......................] - ETA: 51s - loss: 1.1399 - acc: 0.7313

...

50000/50000 [==============================] - 72s 1ms/step - loss: 0.9400 - acc: 0.8095 - val_loss: 0.9093 - val_acc: 0.8198
Epoch 53/100
13152/50000 [======>.......................] - ETA: 50s - loss: 0.9191 - acc: 0.8142

50000/50000 [==============================] - 72s 1ms/step - loss: 0.9447 - acc: 0.8061 - val_loss: 0.9550 - val_acc: 0.7966
Epoch 54/100
12320/50000 [======>.......................] - ETA: 52s - loss: 0.9079 - acc: 0.8170

50000/50000 [==============================] - 72s 1ms/step - loss: 0.9334 - acc: 0.8084 - val_loss: 0.9520 - val_acc: 0.8033
Epoch 55/100
12224/50000 [======>.......................] - ETA: 51s - loss: 0.9026 - acc: 0.8175

50000/50000 [==============================] - 72s 1ms/step - loss: 0.9466 - acc: 0.8058 - val_loss: 0.9446 - val_acc: 0.8039
Epoch 56/100
12736/50000 [======>.......................] - ETA: 50s - loss: 0.9292 - acc: 0.8127

50000/50000 [==============================] - 72s 1ms/step - loss: 0.9439 - acc: 0.8057 - val_loss: 0.9092 - val_acc: 0.8172


In [10]:
from google.colab import files 

In [11]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


In [12]:
files.download(model_path)

In [13]:
model_arch_path = os.path.join(save_dir, 'model_arch.json')
model_json = model.to_json()
with open(model_arch_path, "w") as json_file:
    json_file.write(model_json)
print('Saved model architecture at %s ' % model_arch_path)
files.download(model_arch_path)

Saved model architecture at /content/saved_models/model_arch.json 


In [10]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 338us/step
Test loss: 0.9091593200683594
Test accuracy: 0.8172
